In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

df_rtr = pd.read_csv('produced_tables/rtr.csv', sep='\t', lineterminator='\n', encoding = 'utf8', dtype='unicode')
df_mlt = pd.read_csv('produced_tables/multext.csv', sep='\t', lineterminator='\n', encoding = 'utf8', dtype='unicode')
df_exact = pd.read_csv('produced_tables/stems/only_exact_extracted.csv', sep='\t', lineterminator='\n', encoding = 'utf8', dtype='unicode')

def get_labels(col1_value, col2_value, label1, label2):

    col1_phons = str(col1_value).split(';')
    col2_phons = str(col2_value).split(';')
    label1 = str(label1).split(';')
    label2 = str(label2).split(';')

    if(len(label1) == 1):
        for i in range(1,len(col1_phons)):
            label1.append(label1[0])
    if(len(label2) == 1):
        for i in range(1,len(col2_phons)):
            label2.append(label2[0])

    labels_list = []

    count_1 = 0
    count_2 = 0
    for phons2 in col2_phons:
        phons2 = phons2.split('|')
        for phon2 in phons2:
            for phons1 in col1_phons:
                phons1 = phons1.split('|')
                for phon1 in phons1:
                    try:
                        labels_list.append(label1[count_1] + '_' + label2[count_2])
                    except IndexError:
                        print (label1, count_1, label2, count_2 , col1_phons, col2_phons)
                count_1 += 1
        count_1 = 0
        count_2 += 1
    return ';'.join(labels_list)



labels = ';'.join(df_mlt['Tag'].to_list())

df_rtr['labels_1'] = df_rtr['cat_1']
df_rtr['labels_2'] = df_rtr['cat_2']

df_rtr['labels_1'] = df_rtr['labels_1'].apply(lambda x: labels if x == 'V' else x + ':')
df_rtr['labels_2'] = df_rtr['labels_2'].apply(lambda x: labels if x == 'V' else x+ ':')

exact_list = df_exact['rid'].to_list()

df_rtr = df_rtr.loc[df_rtr['rid'].isin(exact_list)]

#isplay(df_rtr)
#df_rtr[['labels']] = df_rtr.apply(lambda row: pd.Series(get_labels(row['phon_stem_1'], row['phon_stem_2'], row['labels_1'], row['labels_2'])), axis=1)
#df_rtr = df_rtr.drop(columns=['labels_1', 'labels_2'])

df_rtr = df_rtr.drop(columns=['generalization2'])
df_rtr = df_rtr.loc((df_rtr['generalization_1'] != 'X') & (df_rtr['generalization_2'] != 'X'))
df_rtr = df_rtr.loc((df_rtr['generalization_1'] != 'X') & (df_rtr['generalization_2'] != 'Y'))
df_rtr.to_csv('produced_tables/graph_data/data.csv', sep='\t', lineterminator='\n', encoding='utf8', index=False)

display(df_rtr)
display(df_rtr.loc[df_rtr['phon_stem_1'].str.contains('\|')])



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().